# Enter the Query and Get the Top 5 Most Relevant Reviews 

In [2]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

# 1) Load the local model (no download)
model = SentenceTransformer('models/all-MiniLM-L6-v2')

# 2) Load embeddings and dataframe
embeddings = np.load('review_embeddings.npy')
df = pd.read_pickle('reviews_df.pkl')

print("✅ Model and embeddings loaded successfully!\n")

while True:
    # 3) Ask the user for a query
    query = input("Enter your search query (or 'exit' to quit): ").strip()
    if query.lower() == 'exit':
        print("👋 Exiting search. Goodbye!")
        break

    # 4) Encode the query
    query_embedding = model.encode(query)

    # 5) Compute cosine similarities
    scores = util.cos_sim(query_embedding, embeddings)[0].cpu().numpy()

    # 6) Get top-k results
    topk = 5
    topk_indices = np.argsort(-scores)[:topk]

    # 7) Display results
    print(f"\nTop {topk} results for: '{query}'\n")
    for idx in topk_indices:
        print(f"🔹 Score: {scores[idx]:.4f}")
        print(f"💬 Review: {df.iloc[idx]['reviewText']}\n")

    print("-" * 60 + "\n")


✅ Model and embeddings loaded successfully!


Top 5 results for: 'lightroom'

🔹 Score: 0.6113
💬 Review: I am a frequent user of Adobe products such as PhotoShop and Lightroom. I found this product a little bit difficult to begin using. It took me a bit of time to find everything I wanted. The performance was not too bad, and I was able to do everything I normally use Lightroom for. If I had a choice, I'd much rather use Lightroom. It's easier (perhaps just because that's what I'm used to?) and does seem to run a tad faster and smoother. That being said, this product retails for significantly less than Lightroom. If you cannot afford Lightroom or are on a tight budget, this is a perfectly good program to purchase instead. It seems to offer the same functionality as long as you're willing to be a it patient with it.

🔹 Score: 0.5992
💬 Review: This software does almost everything that Lightroom does at a less cost. Most people do not need or use all the features in Lightroom. I am not a p